In [1]:
#필요한 것들 install
!pip3 install pyperclip
!pip3 install selenium

  Created wheel for pyperclip: filename=pyperclip-1.8.0-py3-none-any.whl size=8691 sha256=f999b8155bd1fa3ead6c75dc11dab031e67f34e5b69f570d3b9b64e85c9190ed
  Stored in directory: /Users/jayyoon/Library/Caches/pip/wheels/03/79/58/ab51f0f590281b0f4f6046d9271eb98cc50565afb9200f155a
Successfully built pyperclip


In [120]:
import pandas as pd
import numpy as np

import re
import requests # urllib 말고 requests 써주세요 (urllib은 구식입니다!)
from bs4 import BeautifulSoup as bs
import lxml
from selenium import webdriver # 자바스크립트로 전송 되는 데이터는 url - html에 안붙어있습니다.. 페이지를 열어줘야 데이터가 와요!
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

from collections import OrderedDict
from itertools import repeat
import pyperclip

In [103]:
import os
os.getcwd()

'/Users/jayyoon'

In [122]:
#chromedriver에 연동
path = '/Users/jayyoon/Desktop/chromedriver'
driver = webdriver.Chrome(path)
driver.implicitly_wait(3)

In [ ]:
#요기요 홈페이지 접속(일단은 주소 정해서)
driver.get('https://www.yogiyo.co.kr/mobile/#/')
driver.implicitly_wait(3)

In [124]:
#주소 입력
address = '서울특별시 광진구 뚝섬로 34길 9'
driver.find_element_by_xpath('//*[@id="search"]/div/form/input').send_keys(address)

In [125]:
#검색 클릭
driver.find_element_by_xpath('//*[@id="button_search_address"]/button[2]').click()

#전체보기 클릭
driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/a').click()

In [126]:
#첫 번째 음식점 클릭
#다른 음식점의 경우, 맨 마지막 div[]안에 숫자를 바꿔주면 된다 
#ex) 두번째: //*[@id="content"]/div/div[4]/div/div[2]/div[2]/div
driver.find_element_by_xpath('//*[@id="content"]/div/div[3]/div/div[2]/div[1]/div').click()

In [280]:
#data 저장할 데이터프레임 생성
food_list = pd.DataFrame(columns = ['restaurant', 'category', 'menu', 'original price', 'discounted price'])
food_list

,restaurant,category,menu,original price,discounted price


In [281]:
name = bs(driver.page_source, 'html.parser').find('span', {'class': 'restaurant-name ng-binding'}).text
for i in bs(driver.page_source, 'html.parser').find_all('div', {'class': 'panel panel-default ng-scope'},
                                              'category in restaurant.menu'):
    for j in i.find_all('span', {'class': 'menu-name pull-left ng-binding pop-menu'}):
        if j.text == '인기메뉴':
            for k in i.find_all('div', {'class': 'menu-name ng-binding'}):
                category = j.text
                menu = k.text
                food_list = food_list.append({'restaurant': name, 'menu': k.text, 'category': j.text},
                                              ignore_index = True)         

In [291]:
for i in bs(driver.page_source, 'html.parser').find_all('div', {'class': 'panel panel-default ng-scope'},
                                                       'category in restaurant.menu'):
    for j in i.find_all('span', {'class': 'menu-name pull-left ng-binding pop-menu'}):
        for k in i.find_all('div', {'class': menu})

<span class="menu-name pull-left ng-binding pop-menu" ng-class="get_menu_class(category.slug)">인기메뉴</span>


In [284]:
food_list.drop_duplicates(['restaurant', 'menu'])

,restaurant,category,menu,original price,discounted price
0,셰플리-셰프의건강한한끼(영등포),인기메뉴,수원왕갈비 치밥,NaN,NaN
1,셰플리-셰프의건강한한끼(영등포),인기메뉴,두툼한 통등심 가츠동,NaN,NaN
2,셰플리-셰프의건강한한끼(영등포),인기메뉴,매콤대패삼겹덮밥,NaN,NaN
3,셰플리-셰프의건강한한끼(영등포),인기메뉴,씨푸드 나시고랭,NaN,NaN
4,셰플리-셰프의건강한한끼(영등포),인기메뉴,그라나파다노 김치볶음밥,NaN,NaN


In [258]:
#일반메뉴 추가(여기도 똑같을 것으로 예상)
for i in bs(driver.page_source, 'html.parser').find_all('div', {'class': 'panel panel-default ng-scope'},
                                              'category in restaurant.menu'):
    for j in i.find_all('span', {'class': 'menu-name pull-left ng-binding'}):
        if j.text == 'Photo Menu Items': continue #photo menu 빼고
        if '[요기서결제 시]' in j.text: continue #[요기서결제 시] 빼고
        for k in i.find_all('div', {'class' : 'menu-name ng-binding'}):
            food_list = food_list.append({'restaurant': name,'menu': k.text, 'category': j.text}, ignore_index = True)

버거
세트
디저트
드링크
치킨
팩류
아이스크림류


In [ ]:
def crawling(paeg_source, 

In [261]:
#중복행 제거(두번씩 나오는 문제 해결 + 인기메뉴에 있으면 그 메뉴 일반메뉴에서 제거)
food_list = food_list.drop_duplicates(['restaurant', 'menu', 'original price', 'discounted price'])
food_list

,restaurant,category,menu,original price,discounted price
0,7번가피자-천호점,인기메뉴,슈퍼콤비네이션 R,NaN,NaN
1,7번가피자-천호점,인기메뉴,홍대피자 R,NaN,NaN
2,7번가피자-천호점,인기메뉴,흥부 포테이토 R,NaN,NaN
3,7번가피자-천호점,인기메뉴,샘스테이크 R,NaN,NaN
4,7번가피자-천호점,인기메뉴,슈퍼콤비네이션 L,NaN,NaN
...,...,...,...,...,...
247,롯데리아-대방역점,팩류,NEW홈투게더팩（사이다）,NaN,NaN
254,롯데리아-대방역점,아이스크림류,롯데리아찐氷,NaN,NaN
255,롯데리아-대방역점,아이스크림류,토네이도초코쿠키,NaN,NaN
256,롯데리아-대방역점,아이스크림류,토네이도녹차,NaN,NaN


In [262]:
food_list[food_list['restaurant'] == '7번가피자-천호점'] 

,restaurant,category,menu,original price,discounted price
0,7번가피자-천호점,인기메뉴,슈퍼콤비네이션 R,NaN,NaN
1,7번가피자-천호점,인기메뉴,홍대피자 R,NaN,NaN
2,7번가피자-천호점,인기메뉴,흥부 포테이토 R,NaN,NaN
3,7번가피자-천호점,인기메뉴,샘스테이크 R,NaN,NaN
4,7번가피자-천호점,인기메뉴,슈퍼콤비네이션 L,NaN,NaN
...,...,...,...,...,...
62,7번가피자-천호점,사이드,디핑소스,NaN,NaN
63,7번가피자-천호점,사이드,시즈닝포테이토,NaN,NaN
64,7번가피자-천호점,사이드,핫소스,NaN,NaN
65,7번가피자-천호점,사이드,치즈가루,NaN,NaN


In [263]:
food_list[food_list['restaurant'] == '셰플리-셰프의건강한한끼(영등포)'] 

,restaurant,category,menu,original price,discounted price
67,셰플리-셰프의건강한한끼(영등포),인기메뉴,수원왕갈비 치밥,NaN,NaN
68,셰플리-셰프의건강한한끼(영등포),인기메뉴,두툼한 통등심 가츠동,NaN,NaN
69,셰플리-셰프의건강한한끼(영등포),인기메뉴,매콤대패삼겹덮밥,NaN,NaN
70,셰플리-셰프의건강한한끼(영등포),인기메뉴,씨푸드 나시고랭,NaN,NaN
71,셰플리-셰프의건강한한끼(영등포),인기메뉴,그라나파다노 김치볶음밥,NaN,NaN
72,셰플리-셰프의건강한한끼(영등포),1인분주문,순살 새우치킨 콤보（반반）,NaN,NaN
73,셰플리-셰프의건강한한끼(영등포),1인분주문,명란마요 가라아게 콤보,NaN,NaN
74,셰플리-셰프의건강한한끼(영등포),1인분주문,직화 유자삼겹 분짜,NaN,NaN
75,셰플리-셰프의건강한한끼(영등포),1인분주문,모둠나물 제육비빔밥,NaN,NaN
76,셰플리-셰프의건강한한끼(영등포),1인분주문,소불고기 비빔밥,NaN,NaN


In [264]:
food_list[food_list['restaurant'] == '롯데리아-대방역점'] 

,restaurant,category,menu,original price,discounted price
84,롯데리아-대방역점,인기메뉴,불고기버거세트,NaN,NaN
85,롯데리아-대방역점,인기메뉴,새우버거세트,NaN,NaN
86,롯데리아-대방역점,인기메뉴,핫크리스피버거세트,NaN,NaN
87,롯데리아-대방역점,인기메뉴,모짜렐라인더 버거-베이컨 세트,NaN,NaN
88,롯데리아-대방역점,인기메뉴,새우버거,NaN,NaN
...,...,...,...,...,...
247,롯데리아-대방역점,팩류,NEW홈투게더팩（사이다）,NaN,NaN
254,롯데리아-대방역점,아이스크림류,롯데리아찐氷,NaN,NaN
255,롯데리아-대방역점,아이스크림류,토네이도초코쿠키,NaN,NaN
256,롯데리아-대방역점,아이스크림류,토네이도녹차,NaN,NaN


In [279]:
#식당, 카테고리, 메뉴 이름만 우선
food_list.to_csv('/Users/jayyoon/Desktop/exmaple.csv')

In [121]:
#chromedriver 종료
driver.close()

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=85.0.4183.83)
